<a href="https://colab.research.google.com/github/Amarnath-188/Design-Optimization/blob/Project-1/Inverted_pendulum_MPC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Introduction**

This inverted pendulum thing on a cart that can move back and forth. The challenge is to create a smart controller, using something called Model Predictive Control (MPC), to guide the cart. The goal is to make the cart go from a starting point at  y = -1.0  to  y = 0.0  in just 8 seconds. And, here's the catch – we want to double-check that the speed of the cart v , the tilt of the pendulum  $\theta$, and this adjustable cart thing  q  are all at zero both before and after the move. This MPC thing is cool because it predicts how the system will behave and adjusts the controls accordingly. Basically, it's like teaching the system to do a dance to get from point A to point B without falling over, and we want it to nail the routine in under 6.2 seconds. Oh, and make sure everything's calm and at rest when it's done.


# **Here's a breakdown of the key components:**

1. **System Description**: The system consists of an inverted pendulum mounted on an adjustable cart. The cart can move horizontally along a track, and the objective is to control the motion of the cart in such a way that the pendulum remains stable in an upright position.

2. **Controller Type**: Model Predictive Control (MPC) is the chosen control strategy. MPC is an advanced control technique that uses a dynamic model of the system to predict its future behavior and optimize control inputs over a finite time horizon.

3. **Maneuver Requirements**: The cart needs to execute a sequence of moves that result in its position changing from  y = -1.0  to  y = 0.0 within 6.2 seconds. This implies the need for a carefully tuned control strategy that considers the dynamics of the system and the time constraints.

4. **Parameter Verification**: Before and after the maneuver, it is essential to verify that certain parameters are zero. This includes the velocity of the cart  v , the angle of the pendulum $\theta$, and the adjustable parameter for the cart q. Ensuring these parameters are zero ensures the stability and desired state of the system.

5. **Time Constraint**: The maneuver must be completed within a specific time frame 8 seconds, adding a temporal aspect to the control problem.

In summary, the problem involves designing a Model Predictive Controller to control the motion of an inverted pendulum system with an adjustable cart. The controller should be capable of executing a predefined sequence of moves to transition the cart from one position to another within a specified time, while ensuring the stability of the system by verifying key parameters before and after the maneuver.



$\Lambda_{12}= \frac{V2}{V1} e^{\frac{-\lambda_{12}}{R_g T}}$

$\Lambda_{21}= \frac{V1}{V2} e^{\frac{-\lambda_{21}}{R_g T}}$

$\ln(\gamma_1) = -\ln(x_1 + \Lambda_{12} x_2) + x_2(\frac{\Lambda_{12}}{x_1 + \Lambda_{12} x_2}-\frac{\Lambda_{21}}{x_2 + \Lambda_{21} x_1})$

$\ln(\gamma_2) = -\ln(x_2 + \Lambda_{21} x_1) - x_1(\frac{\Lambda_{12}}{x_1 + \Lambda_{12} x_2}-\frac{\Lambda_{21}}{x_2 + \Lambda_{21} x_1})$


Correlations for the vapor pressure $P_1^{sat}$ and density (rho) of numerous commonly encountered pure components can be found in BYU's DIPPR database. In this situation, $P_1^{sat}$ varies with temperature in accordance with a specified function.

$P_{sat} = e^{(A_o + \frac{B_o}{T}+ C_o \ln(T)+D_o T^{E_o})}$

$\rho = \frac{A}{(1+1-{\frac{T}{C}})}$

$v = 1/\rho$

$\rho$ and $v$ is also a function of Temperature

The formula for determining the degrees of freedom (DOF) in a system involving multiple components and phases is expressed as DOF = 2 + #Components - #Phases. Here, the system consists of two phases (liquid and vapor) and two components (ethanol and cyclohexane), resulting in two degrees of freedom that require specification. In this scenario, a choice must be made to constrain two out of the four measured values for the system, involving x1, y1, P, or T. It is advisable to constrain the values of x1 and P, as demonstrated in the tutorial below.

In [ ]:
pip install gekko

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 85.9 MB/s eta 0:00:00


In [ ]:
# importing libraries
from gekko import GEKKO
from scipy.optimize import fsolve
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Molar gas constant
R = 1.9858775  # cal/mol K

def vaporP(T: float, compound: list):
    '''
    T              ---> temp K
    param compound ---> vapor pressure
    return         ---> Vapor Pressure in Pa
    '''
    VP = np.exp(compound[0] + compound[1]/T + compound[2]*np.log(T) + compound[3]*T**compound[4])
    return VP

In [ ]:
def liquidMolarVolume(T: float, mvolume: list):
    '''
    T               ---> temp in K
    mvolume         ---> component liquid molar volume
    return          ---> liquid molar volume
    '''
    L_mol_volume = 1 / mvolume[0] * mvolume[1] ** (1 + (1 - T / mvolume[2]) ** mvolume[3])
    return L_mol_volume

In [ ]:
def Lij(aij: float, T: float, Vlvali: float,\
            Vlvalj: float):
    '''
    # Vl1 = liquidMolarVolume(T, Vlvali)
    # Vl2 = liquidMolarVolume(T, Vlvalj)
    '''
    Vl1 = Vlvali
    Vl2 = Vlvalj
    V = Vl2 / Vl1 * np.exp(-aij / (R * T))
    return V

In [ ]:
def Gamma(a12: float, a21: float, x1: float,\
          Temp: float, Compound: int):


  '''
  Compound 1: Ethanol
  Compound 2: Cyclohexane
  Liquid molar volume
  yi * P =  γi * xi * Pisat
  '''

  Vethanol_mvol = 0.058492
  VCyclochexane_mvol = 0.10882

  x2 = 1.0 - x1
  L12 = Lij(a12, Temp, Vethanol_mvol, VCyclochexane_mvol)
  L21 = Lij(a21, Temp,  VCyclochexane_mvol, Vethanol_mvol)
  L11 = 1.0
  L22 = L11

  if Compound == 1:
      A = (x2 * L12)+x1
      B = x1 * L11 / (x1 * L11 + x2 * L12) \
           +x2 * L21 / (x1 * L21 + x2 * L22)

  elif Compound == 2:

      A = (x1 * L21)+x2
      B = x2 * L22 / (x2 * L22 + x1 * L21) \
         +x1 * L12 / (x2 * L12 + x1 * L11)

  C = np.exp(1.0-np.log(A)-B)

  return C